In [1]:
!pip install transformers pipeline datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
from transformers import pipeline
from datasets import load_dataset
import torch
import pandas as pd

In [3]:
roberta = pipeline(model="deepset/roberta-base-squad2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [4]:
# test with actual model
from transformers import AutoTokenizer, RobertaForQuestionAnswering

# initialize model and tokenizer for roberta model for QA
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [5]:
# method to call model, get prediction and return output
def get_answer(question, context):
  inputs = tokenizer(question, context, return_tensors="pt", truncation=True)
  outputs = model(**inputs)
  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()
  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
  return answer

### Testing over-extraction cases

In [ ]:
context = 'Some theories of civil disobedience hold that civil disobedience is only justified against governmental entities. Brownlee argues that disobedience in opposition to the decisions of non-governmental agencies such as trade unions, banks, and private universities can be justified if it reflects "a larger challenge to the legal system that permits those decisions to be taken". The same principle, she argues, applies to breaches of law in protest against international organizations and foreign governments.'

ds_question = "Who claims that public companies can also be part of civil disobedience?"
question = "Who claims that non-governmental agencies can also be part of civil disobedience?"

print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")
print(f"Modified Question Prediction: {get_answer(question, context)}")

Dataset Question Prediction: Who claims that public companies can also be part of civil disobedience?Some theories of civil disobedience hold that civil disobedience is only justified against governmental entities. Brownlee

Modified Question Prediction:  Brownlee


In [ ]:
context = "Terra preta (black earth), which is distributed over large areas in the Amazon forest, is now widely accepted as a product of indigenous soil management. The development of this fertile soil allowed agriculture and silviculture in the previously hostile environment; meaning that large portions of the Amazon rainforest are probably the result of centuries of human management, rather than naturally occurring as has previously been supposed. In the region of the Xingu tribe, remains of some of these large settlements in the middle of the Amazon forest were found in 2003 by Michael Heckenberger and colleagues of the University of Florida. Among those were evidence of roads, bridges and large plazas."

ds_question = "What did the development of this fertile soil provide in hostile environment?"
question = "What did the development of this fertile soil allow in hostile environment?"

print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")
print(f"Modified Question Prediction: {get_answer(question, context)}")

Dataset Question Prediction: What did the development of this fertile soil provide in hostile environment?Terra preta (black earth), which is distributed over large areas in the Amazon forest, is now widely accepted as a product of indigenous soil management. The development of this fertile soil allowed agriculture and silviculture

Modified Question Prediction:  agriculture and silviculture


### Contextual Misunderstanding - Numerical and Temporal

In [ ]:
context = "This was the first Super Bowl to feature a quarterback on both teams who was the #1 pick in their draft classes. Manning was the #1 selection of the 1998 NFL draft, while Newton was picked first in 2011. The matchup also pits the top two picks of the 2011 draft against each other: Newton for Carolina and Von Miller for Denver. Manning and Newton also set the record for the largest age difference between opposing Super Bowl quarterbacks at 13 years and 48 days (Manning was 39, Newton was 26)."

ds_question = "How much older was Manning than Newton during Super Bowl 50?"
question = "What was the age difference between Manning and Newton during Super Bowl 50?"

print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")
print(f"Modified Question Prediction: {get_answer(question, context)}")

Dataset Question Prediction:  39

Modified Question Prediction:  13 years and 48 days


In [ ]:
context = "The Yuan dynasty is considered both a successor to the Mongol Empire and an imperial Chinese dynasty. It was the khanate ruled by the successors of Möngke Khan after the division of the Mongol Empire. In official Chinese histories, the Yuan dynasty bore the Mandate of Heaven, following the Song dynasty and preceding the Ming dynasty. The dynasty was established by Kublai Khan, yet he placed his grandfather Genghis Khan on the imperial records as the official founder of the dynasty as Taizu.[b] In the Proclamation of the Dynastic Name (《建國號詔》), Kublai announced the name of the new dynasty as Great Yuan and claimed the succession of former Chinese dynasties from the Three Sovereigns and Five Emperors to the Tang dynasty."

ds_question = "What dynasty came after the Yuan?"
#question = "What was the age difference between Manning and Newton during Super Bowl 50?"

print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")
#print(f"Modified Question Prediction: {get_answer(question, context)}")

Dataset Question Prediction:  Song dynasty



### Contextual Misunderstanding - Shared Phrase

In [6]:
context = "Euglenophytes are a group of common flagellated protists that contain chloroplasts derived from a green alga. Euglenophyte chloroplasts have three membranes—it is thought that the membrane of the primary endosymbiont was lost, leaving the cyanobacterial membranes, and the secondary host's phagosomal membrane. Euglenophyte chloroplasts have a pyrenoid and thylakoids stacked in groups of three. Starch is stored in the form of paramylon, which is contained in membrane-bound granules in the cytoplasm of the euglenophyte."

ds_question = "What kind of chloroplasts do Euglenophytes have?"


print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")

Dataset Question Prediction:  three membranes



In [7]:
context = "Much of the work of the Scottish Parliament is done in committee. The role of committees is stronger in the Scottish Parliament than in other parliamentary systems, partly as a means of strengthening the role of backbenchers in their scrutiny of the government and partly to compensate for the fact that there is no revising chamber. The principal role of committees in the Scottish Parliament is to take evidence from witnesses, conduct inquiries and scrutinise legislation. Committee meetings take place on Tuesday, Wednesday and Thursday morning when Parliament is sitting. Committees can also meet at other locations throughout Scotland."

ds_question = "Taking evidence from witnesses is one of committees' what?"


print(f"Dataset Question Prediction: {get_answer(ds_question, context)}\n")

Dataset Question Prediction:  conduct inquiries and scrutinise legislation

